In [1]:
import re
import pandas as pd
import openpyxl
from pathlib import Path
from tabulate import tabulate

In [7]:
# CAN-ID mapping
CAN_ID_MAP = {
    "0x1FC017FF".lower(): ("WEZ", 2),
    "0x1FC00FFF".lower(): ("WEZ", 1),
}

In [ ]:
log_file = "logs_esp32-eth02_run.txt"  # path to your log file
excel_file = "TTE-GW-Modbus-datapoints_edited.xlsx"  # path to your input Excel file

log_file = Path(log_file)

In [61]:
def extract_datapoints(logfile):
    """Extract DatapointId + CAN ID pairs from log file ([NEW] lines only)."""
    results = []
    with open(logfile, "r") as f:
        for i, line in enumerate(f):
            if "[NEW]" not in line and "[EXS]" not in line:
                continue
            can_match = re.search(r"Can-ID:\s+(0x[0-9A-Fa-f]+)", line)
            dp_match = re.search(r"Datapoint:\s+0x[0-9A-Fa-f]+\s+\((\d+)\)", line)
            group_match = re.search(r"function_group:\s+0x[0-9A-Fa-f]+\s+\((\d+)\)", line)
            number_match = re.search(r"function_number:\s+0x[0-9A-Fa-f]+\s+\((\d+)\)", line)

            if can_match and dp_match and group_match and number_match:
                can_id = can_match.group(1).upper()
                datapoint = int(dp_match.group(1))
                group = int(group_match.group(1))
                number = int(number_match.group(1))
                if can_id.lower() in CAN_ID_MAP.keys():
                    unit_name, unit_id = CAN_ID_MAP.get(can_id.lower(), can_id)
                else:
                    unit_name = can_id
                    unit_id = -1
                results.append((i, unit_name, unit_id, group, number, datapoint))
    return results

results = extract_datapoints(log_file)
results_df = pd.DataFrame(results, columns=["Line", "UnitName", "UnitId", "FunctionGroup", "FunctionNumber", "DatapointId"])
results_df.drop_duplicates(subset=["UnitName", "UnitId", "FunctionGroup", "FunctionNumber", "DatapointId"], inplace=True)
results_df

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId
0,101,WEZ,2,2,0,504
1,151,WEZ,2,2,0,4
2,160,WEZ,1,1,0,500
5,252,WEZ,1,1,0,7036
6,253,WEZ,1,1,1,500
...,...,...,...,...,...,...
241,1708,0X1E8017FF,-1,10,1,2082
248,1753,WEZ,1,1,1,1002
253,1773,WEZ,2,10,1,9075
259,1828,WEZ,2,10,1,20053


In [ ]:
results_df.sort_values(by=["DatapointId"])

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId
234,1593,WEZ,2,0,0,0
64,631,WEZ,1,0,0,0
48,569,WEZ,1,60,254,0
58,605,WEZ,1,60,254,1
50,577,WEZ,1,1,1,1
...,...,...,...,...,...,...
220,1491,WEZ,2,10,1,23085
175,1224,0X1E800FFF,-1,10,1,29050
211,1419,WEZ,1,0,0,38003
135,1051,WEZ,1,0,0,38012


In [50]:
df_ref = pd.read_excel(excel_file, sheet_name=1)
df_ref

,Register Address,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,DatapointName,Type,TypeName,Decimal,...,Text 22,Text 23,Text 24,Text 25,Text 26,Text 27,Text 28,Text 29,Text 30,Text 31
0,1,SOL,65,22,0,14,TKO1 Kollektor Temperatur,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,SOL,65,22,1,14,TKO2 Kollektor Temperatur,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,SOL,65,22,1,2034,Gesamtertrag Kollektor_high,2,S32,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,SOL,65,22,1,2034,Gesamtertrag Kollektor_low,1,S32,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,SOL,65,22,1,2030,Kollektorleistung aktuell,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30929,31767,PS,143,21,0,6055,Sollw.-Reduktion Puffer-Sollwert Kühlen,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30930,31768,PS,142,21,0,6033,Puffer-Sollwert bei Konstantanf.Heizen,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30931,31769,PS,142,21,0,6056,Puffer-Sollwert bei Konstantanf. Kühlen,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30932,31770,PS,143,21,0,6033,Puffer-Sollwert bei Konstantanf.Heizen,1,S16,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_test = pd.merge(results_df, df_ref, on=["UnitName", "UnitId", "FunctionGroup", "FunctionNumber", "DatapointId"], how="right", suffixes=('_log', '_ref'))
df_test

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,Register Address,DatapointName,Type,TypeName,...,Text 22,Text 23,Text 24,Text 25,Text 26,Text 27,Text 28,Text 29,Text 30,Text 31
0,NaN,SOL,65,22,0,14,1,TKO1 Kollektor Temperatur,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,SOL,65,22,1,14,2,TKO2 Kollektor Temperatur,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,SOL,65,22,1,2034,3,Gesamtertrag Kollektor_high,2,S32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,SOL,65,22,1,2034,4,Gesamtertrag Kollektor_low,1,S32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SOL,65,22,1,2030,5,Kollektorleistung aktuell,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30929,NaN,PS,143,21,0,6055,31767,Sollw.-Reduktion Puffer-Sollwert Kühlen,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30930,NaN,PS,142,21,0,6033,31768,Puffer-Sollwert bei Konstantanf.Heizen,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30931,NaN,PS,142,21,0,6056,31769,Puffer-Sollwert bei Konstantanf. Kühlen,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30932,NaN,PS,143,21,0,6033,31770,Puffer-Sollwert bei Konstantanf.Heizen,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df_out = df_test[~df_test["Line"].isna()].sort_values(by=["DatapointId"])
df_out

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,Register Address,DatapointName,Type,TypeName,...,Text 22,Text 23,Text 24,Text 25,Text 26,Text 27,Text 28,Text 29,Text 30,Text 31
1376,631.0,WEZ,1,0,0,0,1477,AF1 - Aussenfühler 1,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,569.0,WEZ,1,60,254,0,1520,Sollwert für Heizkreisbetrieb,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1509,1593.0,WEZ,2,0,0,0,1610,AF1 - Aussenfühler 1,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,577.0,WEZ,1,1,1,1,1511,Raum-Ist,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,605.0,WEZ,1,60,254,1,1521,Sollwert für Speicherbetrieb,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,869.0,WEZ,1,10,1,23085,1564,Emissionstest aktivieren,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,1491.0,WEZ,2,10,1,23085,1697,Emissionstest aktivieren,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26704,1419.0,WEZ,1,0,0,38003,27542,SG Mindestleistung,1,U16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26707,1051.0,WEZ,1,0,0,38012,27545,Smart Grid über Systembus,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_wez1 = df_out[(df_out["UnitName"] == "WEZ") & (df_out["UnitId"] == 1)]
df_wez1

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,Register Address,DatapointName,Type,TypeName,...,Text 22,Text 23,Text 24,Text 25,Text 26,Text 27,Text 28,Text 29,Text 30,Text 31
1376,631.0,WEZ,1,0,0,0,1477,AF1 - Aussenfühler 1,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,569.0,WEZ,1,60,254,0,1520,Sollwert für Heizkreisbetrieb,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,577.0,WEZ,1,1,1,1,1511,Raum-Ist,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,605.0,WEZ,1,60,254,1,1521,Sollwert für Speicherbetrieb,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,573.0,WEZ,1,1,0,1,1510,Raum-Ist,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26629,1199.0,WEZ,1,10,1,23008,27467,JAZ Jahresarbeitszahl WEZ,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1463,869.0,WEZ,1,10,1,23085,1564,Emissionstest aktivieren,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26704,1419.0,WEZ,1,0,0,38003,27542,SG Mindestleistung,1,U16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26707,1051.0,WEZ,1,0,0,38012,27545,Smart Grid über Systembus,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
print("".join(df_wez1[["Register Address", "DatapointName"]].apply(lambda x : "{}, # {}\r\n".format(*x.astype(str)),1)))

1477, # AF1 - Aussenfühler 1
1520, # Sollwert für Heizkreisbetrieb
1511, # Raum-Ist
1521, # Sollwert für Speicherbetrieb
1510, # Raum-Ist
1515, # Vorlauf-Ist
1514, # Vorlauf-Ist
1513, # Vorlauf-Ist
1500, # Warmwasser-Ist SF
18725, # Wärmeerzeuger-Ist
18742, # Rücklauftemperatur Wärmeerzeuger
1525, # WEZ-Temperatur
1529, # Heizung Sollwert
18740, # Drehzahl Hauptpumpe
1530, # Speicher Sollwert
1531, # WEZ Sollwert
1534, # Fehlercode vom Automaten
1536, # WEZ-Leistung
1537, # Absolute Leistung
1539, # WEZ-Status
1540, # Betriebsstatus
27490, # Coefficient of Performance
27491, # Sondenvorlauf-/Ansaugtemp.
27492, # Sondenrücklauf-/Verdampferoberfl.temp.
18760, # Anlage Vorlauftemperatur Heizen
19868, # Anlage Vorlauftemperatur WW
1494, # Raum-Soll
1493, # Raum-Soll
19563, # Vorlauf-Soll
19562, # Vorlauf-Soll
1499, # Warmwasser-Soll
18724, # Wärmeerzeuger-Soll
18769, # Sollwert Leistung Wärmeerzeuger
18745, # Sollwert Leistung Wärmeerzeuger
18767, # Sollwert Leistung Wärmeerzeuger
19706, #

In [88]:
df_wez2 = df_out[(df_out["UnitName"] == "WEZ") & (df_out["UnitId"] == 2)]
df_wez2

,Line,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,Register Address,DatapointName,Type,TypeName,...,Text 22,Text 23,Text 24,Text 25,Text 26,Text 27,Text 28,Text 29,Text 30,Text 31
1509,1593.0,WEZ,2,0,0,0,1610,AF1 - Aussenfühler 1,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1532,151.0,WEZ,2,2,0,4,1633,Warmwasser-Ist SF,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26715,617.0,WEZ,2,2,0,6,27553,Warmwasser-Ist SF2,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17948,361.0,WEZ,2,10,1,7,18786,Wärmeerzeuger-Ist,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17965,914.0,WEZ,2,10,1,8,18803,Rücklauftemperatur Wärmeerzeuger,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568,1111.0,WEZ,2,60,254,30,1669,WEZ-Leistung,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1569,1115.0,WEZ,2,60,254,31,1670,Absolute Leistung,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26722,1180.0,WEZ,2,60,254,45,27560,Coefficient of Performance,1,U8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1531,358.0,WEZ,2,2,0,1004,1632,Warmwasser-Soll,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17947,888.0,WEZ,2,10,1,1007,18785,Wärmeerzeuger-Soll,1,S16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
print("".join(df_wez2[["Register Address", "DatapointName"]].apply(lambda x : "{}, # {}\r\n".format(*x.astype(str)),1)))

1610, # AF1 - Aussenfühler 1
1633, # Warmwasser-Ist SF
27553, # Warmwasser-Ist SF2
18786, # Wärmeerzeuger-Ist
18803, # Rücklauftemperatur Wärmeerzeuger
1669, # WEZ-Leistung
1670, # Absolute Leistung
27560, # Coefficient of Performance
1632, # Warmwasser-Soll
18785, # Wärmeerzeuger-Soll
19757, # SLP Warmwasser-Ladepumpe
1629, # Betriebswahl Warmwasser
1694, # Betriebswahl Wärmeerzeuger
18787, # Modulation
18798, # Betriebsmeldung
1679, # MK1 HW-Ausgang
1680, # YK1+ HW-Ausgang
1681, # YK1- HW-Ausgang
1683, # SLP HW-Ausgang
1685, # VA2 HW-Ausgang
19054, # Info 1 - vermutlich DF1-1 (21-015)
19055, # Info 2 - vermutlich DF1-2 (21-018)
25615, # Aktuelle elektr. Leistungsaufnahme WEZ
25616, # Aktuelle therm. Leistungsaufnahme WEZ
27468, # JAZ Jahresarbeitszahl WEZ
1697, # Emissionstest aktivieren



In [97]:
pd.set_option("display.max_rows", None)

In [98]:
display(df_out.sort_values(by=["DatapointId"])[["UnitName", "UnitId", "FunctionGroup", "FunctionNumber", "DatapointId", "DatapointName", "Register Address", "FunctionGroup name", "Function name"]])

,UnitName,UnitId,FunctionGroup,FunctionNumber,DatapointId,DatapointName,Register Address,FunctionGroup name,Function name
1376,WEZ,1,0,0,0,AF1 - Aussenfühler 1,1477,Allgemein,Allgemein
1419,WEZ,1,60,254,0,Sollwert für Heizkreisbetrieb,1520,Automat,FA Allgemein
1509,WEZ,2,0,0,0,AF1 - Aussenfühler 1,1610,Allgemein,Allgemein
1410,WEZ,1,1,1,1,Raum-Ist,1511,Heizkreis,Heizkreis 2
1420,WEZ,1,60,254,1,Sollwert für Speicherbetrieb,1521,Automat,FA Allgemein
1409,WEZ,1,1,0,1,Raum-Ist,1510,Heizkreis,Heizkreis 1
1414,WEZ,1,1,2,2,Vorlauf-Ist,1515,Heizkreis,Heizkreis 3
1413,WEZ,1,1,1,2,Vorlauf-Ist,1514,Heizkreis,Heizkreis 2
1412,WEZ,1,1,0,2,Vorlauf-Ist,1513,Heizkreis,Heizkreis 1
1532,WEZ,2,2,0,4,Warmwasser-Ist SF,1633,Warmwasser,Warmwasser 1


In [67]:
df_test.to_excel("df_out.xlsx", index=False)